In [26]:
import json
import datetime

from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage # AIMessage, ToolMessage (not explicitly used in this simplified version)
from langchain.output_parsers import PydanticOutputParser

from datetime import datetime
from IPython.display import display, JSON, Markdown, Image

# from langchain_core.pydantic_v1 import BaseModel, Field # Simplified parsing won't use Pydantic tools for this version

In [2]:
# from src.config import Config
from src.mock_apis import room_services, booking_services
from src.schemas import AgentState, BookingRequest
from src.helper import get_llm, apply_prompt_template

In [16]:
def parse_request(state: AgentState) -> AgentState:
    print("---NODE: PARSE REQUEST---")
    # initialize parser
    parser = PydanticOutputParser(pydantic_object=BookingRequest)
    # apply my predefined prompt template
    prompt_template = apply_prompt_template(parser)
    # get the current date and time
    current_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    # define LLM and the Chain
    llm = get_llm(name="groq")
    chain = prompt_template | llm | parser
    parsed_data = chain.invoke({"user_request": state["original_request"],
                                    "current_date": current_date})
    print("Parsed data:", parsed_data)
    state.update({
            "parsed_request": parsed_data.model_dump(),
            "clarification_needed": parsed_data.clarification_needed,
            "clarification_question": parsed_data.clarification_question,
            "user_name_for_booking": parsed_data.user_name,
            ""
            "error_message": None
        })
        
    return state

In [4]:
# Check if request is valid and clear
def is_clear_request(state: AgentState) -> str:
    print("---CONDITION: Check Valid and Clear Request---")
    if state.get("error_message"):
        print("----> Error detected")
        return "handle_error"
    elif state.get("clarification_needed"):
        print("----> Clarification needed")
        return "ask_clarification"
    else:
        print("----> Ready to book room")
        return "book_room"

In [5]:
def ask_clarification(state: AgentState) -> AgentState:
    print("---NODE: ASK CLARIFICATION---")
    state["final_response_to_user"] = state["clarification_question"]
    return state

In [6]:
def book_room(state: AgentState) -> AgentState:
    # In a real workflow, you'd call your booking API here
    state["final_response_to_user"] = "Booking room..."
    return state

In [22]:
def handle_error(state: AgentState) -> AgentState:
    state["final_response_to_user"] = state.get("error_message", "An unknown error occurred.")
    return END

In [23]:
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("parse_request_node", parse_request)
workflow.add_node("ask_clarification_node", ask_clarification)
workflow.add_node("book_room_node", book_room)
workflow.add_node("handle_error_node", handle_error)

# Set entry point
workflow.set_entry_point("parse_request_node")

# Add edges
workflow.add_conditional_edges(
    "parse_request_node",
    is_clear_request,
    {
        "ask_clarification": "ask_clarification_node",
        "book_room": "book_room_node",
        "handle_error": "handle_error_node"
    }
)
workflow.add_edge("ask_clarification_node", END) # Graph pauses for clarification
workflow.add_edge("book_room_node", END)
workflow.add_edge("handle_error_node", END)

# Compile the graph
app = workflow.compile()

In [24]:
result = app.invoke({
    "original_request": 
    "Book a room for 5 people with a projector tomorrow at 3 PM for 2 hours."
})

---NODE: PARSE REQUEST---
Parsed data: start_time='2025-05-13T15:00:00' duration_hours=2 capacity=5 equipments=['projector'] user_name=None clarification_needed=True clarification_question='Could you please provide your name for the booking?'
---CONDITION: Check Valid and Clear Request---
----> Clarification needed
---NODE: ASK CLARIFICATION---


In [25]:
Markdown(result["final_response_to_user"])

Could you please provide your name for the booking?